In [1]:
! pip install top2vec


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
###code to fetch documents from PDF reader

In [ ]:
from top2vec import Top2Vec

# Train the Top2Vec model
print("Training Top2Vec model. This may take a while...")
model = Top2Vec(documents, speed="learn", workers=4)  # Adjust workers based on your system
print("Model training complete.")


In [ ]:
# Get the number of topics
num_topics = model.get_num_topics()
print(f"The model identified {num_topics} topics.")

# Get topic sizes and labels
topic_sizes, topic_nums = model.get_topic_sizes()

# Display top words for each topic
for topic_num in topic_nums:
    # Use get_topics() to get all topic words and their scores
    topics = model.get_topics()
    
    # Access the words and scores for the specific topic
    topic_words = topics[0][topic_num]  # words for the specific topic
    topic_scores = topics[1][topic_num]  # scores for the specific topic
    
    # Print the top 10 words and their corresponding scores
    print(f"\nTopic {topic_num}:")
    print(", ".join(topic_words[:35]))  # Top 10 words
    print(", ".join([f"{score:.4f}" for score in topic_scores[:10]]))  # Top 10 word scores

In [ ]:
####### VISUALIZATION OF THE TOPICS AND THEIR PROXIMITY ####

import umap
import matplotlib.pyplot as plt

# Get document embeddings and topic vectors
document_embeddings = model.document_vectors  # or model.document_embeddings
topic_embeddings = model.topic_vectors

# Reduce dimensions for visualization using UMAP
umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, metric='correlation')
umap_embeddings = umap_model.fit_transform(document_embeddings)

# Plot the documents
plt.figure(figsize=(10, 8))
plt.scatter(umap_embeddings[:, 0], umap_embeddings[:, 1], alpha=0.5, c='blue', s=10)
plt.title("Document Embeddings")
plt.show()

# Reduce dimensions for topic embeddings
umap_topic_embeddings = umap_model.fit_transform(topic_embeddings)

# Plot the topics
plt.figure(figsize=(10, 8))
plt.scatter(umap_topic_embeddings[:, 0], umap_topic_embeddings[:, 1], alpha=0.5, c='red', s=100)
plt.title("Topic Embeddings")
plt.show()

In [ ]:
# Generate and display a word cloud for each topic
for topic_num in range(model.get_num_topics()):
    model.generate_topic_wordcloud(topic_num)

In [ ]:
####SENTIMENT ANALYSIS POTENTIAL CODE (I suggest diving per political party at first for rough idea)#####

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Step 1: Loop through the first 5 topics
for target_topic in range(5):  # Loop through topic 0 to topic 4
    print(f"\nProcessing Topic {target_topic}...")

    # Step 2: Retrieve documents for the current topic
    documents_for_target_topic = []
    for i, topic in enumerate(topic_assignments):
        if topic == target_topic:
            documents_for_target_topic.append(model.documents[i])  # Access the documents from the model
    
    # Step 3: Print the number of documents associated with this topic
    num_documents = len(documents_for_target_topic)
    print(f"Number of documents in Topic {target_topic}: {num_documents}")

    # Step 4: Perform sentiment analysis on the documents for the current topic
    sentiment_scores = []
    for doc in documents_for_target_topic:
        sentiment = analyzer.polarity_scores(doc)  # Get sentiment score
        sentiment_scores.append(sentiment['compound'])  # Use the compound score (range from -1 to 1)

    # Step 5: Display the sentiment scores for the first few documents
    print(f"\nSentiment scores for documents in Topic {target_topic}:")
    for i, score in enumerate(sentiment_scores[:5]):  # Display sentiment scores for the first 5 documents
        print(f"Document {i+1}: Sentiment Score = {score}")

    # Step 6: Plot the sentiment distribution for the current topic
    plt.figure(figsize=(8, 6))
    plt.hist(sentiment_scores, bins=30, color='skyblue', edgecolor='black')
    plt.title(f"Sentiment Distribution for Documents in Topic {target_topic}")
    plt.xlabel("Sentiment Score (compound)")
    plt.ylabel("Frequency")
    plt.grid(True)
    plt.show()